In [1]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd

C:\Users\Sourav Ray\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
main = pd.read_csv("Main.csv")
har_off_lit = pd.read_csv("Harris_County_Offense_Literal.csv")

In [3]:
h1 = har_off_lit.copy()

In [4]:
h1.drop_duplicates(inplace = True)

In [5]:
h1.insert(0,'Key', range(1, 1+len(h1)))

In [6]:
h1

,Key,Offense_Literal (Harris county)
0,1,ABAN/ENDANG CHILD W/INT TO RET22.04
1,2,ABAN/ENDANG CHILD W/INT TO RET22.041
2,3,ABANDON/ENDANG CHILD W/O INT T22.041
3,4,ABANDONING CHILD UNDER 15 22.041
4,5,ABUSE OF OFF CAP>=2500 <30K
...,...,...
1598,1595,WATER POLL ST LIABIL VIOL INDV
1599,1596,WATER POLLUTION 26.212
1600,1597,WEAPON-POSSESSION OF 46.02
1601,1598,WEAPON FREE SCHOOL ZONE-PROH W46.11


In [7]:
m = main.shape[0]

In [8]:
h = har_off_lit.shape[0]

In [9]:
h/m

64.12

In [10]:
main.tail()

,Classification
20,Liquor laws
21,Drunkenness
22,Disorderly conduct
23,VagrancyCurfewloitering violations
24,Impaired driving


In [11]:
main['Classification'] = main['Classification'].str.upper().str.replace(" ","")

In [12]:
har_off_lit['Offense_Literal (Harris county)'] = har_off_lit['Offense_Literal (Harris county)'].str.upper().str.replace(" ","")

In [13]:
main

,Classification
0,MURDER
1,RAPE
2,OTHERSEXUALASSAULT
3,ROBBERY
4,ASSAULT
5,INTIMATEPARTNERVIOLENCE
6,CHILDMALTREATMENT
7,ARSON
8,IMPAIREDDRIVINGCRASH
9,OTHERIMPAIREDDRIVING


In [14]:
har_off_lit.head()

,Offense_Literal (Harris county)
0,ABAN/ENDANGCHILDW/INTTORET22.04
1,ABAN/ENDANGCHILDW/INTTORET22.041
2,ABANDON/ENDANGCHILDW/OINTT22.041
3,ABANDONINGCHILDUNDER1522.041
4,ABUSEOFOFFCAP>=2500<30K


In [15]:
har_off_lit.nunique()

Offense_Literal (Harris county)    1599
dtype: int64

In [16]:
har_off_lit.shape[0]

1603

In [17]:
har_off_lit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1603 entries, 0 to 1602
Data columns (total 1 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Offense_Literal (Harris county)  1603 non-null   object
dtypes: object(1)
memory usage: 12.6+ KB


In [18]:
har_off_lit[har_off_lit.duplicated()]

,Offense_Literal (Harris county)
1493,TRAFFICPRSNBNFTFORCFRAUPROCON
1543,UNLAWDISCLSEINTIMATEVISUAL
1544,UNLAWDISCLSEINTIMATEVISUAL
1594,VIOLATEPROMISETOAPPEAR


In [19]:
har_off_lit = har_off_lit.drop_duplicates()

In [20]:
main_list = main['Classification'].tolist()
har_list = har_off_lit['Offense_Literal (Harris county)'].tolist()
main_list_condensed = []

In [21]:
def unique(a):
    p = ""
    for char in a:
        if char not in p:
            p = p + char
    return p

In [22]:
for i in main_list:
    main_list_condensed.append(unique(i))

main_list_condensed

['MURDE',
 'RAPE',
 'OTHERSXUAL',
 'ROBEY',
 'ASULT',
 'INTMAEPRVOLC',
 'CHILDMATREN',
 'ARSON',
 'IMPAREDVNGCSH',
 'OTHERIMPADVNG',
 'BURGLAY',
 'LARCENYTHF',
 'MOTRVEHICLF',
 'FRAUD',
 'BUYINGSTOLEPR',
 'VANDLISM',
 'WEAPONSCRYIG',
 'PROSTIUNADEG',
 'DRUGPOSEINAL',
 'GAMBLIN',
 'LIQUORAWS',
 'DRUNKES',
 'DISORELYCNUT',
 'VAGRNCYUFEWLOITS',
 'IMPAREDVNG']

In [61]:
#changing ucr_cat to main ucr groups


#MURDE
#ASLT
#RAPE
#ROBEY
#ARSN
#BURGLAY
#FRAUD
#WEPN
#THEF
#PRSTI
#LIQUOR
#DRUG
#MOTRVEHICL
#GMBLIN
#DRNK
#CHLD

b = 'CHILD'
matches = {}
for i in har_list:
    score = fuzz.partial_ratio(i,b)
    matches[i] = score

In [62]:
ucr_match = pd.DataFrame.from_dict(matches.items())
ucr_match.columns = ['Condensed_Off_Lit','Child Scores']
ucr_match

,Condensed_Off_Lit,Child Scores
0,ABAN/ENDANGCHILDW/INTTORET22.04,100
1,ABAN/ENDANGCHILDW/INTTORET22.041,100
2,ABANDON/ENDANGCHILDW/OINTT22.041,100
3,ABANDONINGCHILDUNDER1522.041,100
4,ABUSEOFOFFCAP>=2500<30K,20
...,...,...
1594,WATERPOLLSTLIABILVIOLINDV,40
1595,WATERPOLLUTION26.212,20
1596,WEAPON-POSSESSIONOF46.02,20
1597,WEAPONFREESCHOOLZONE-PROHW46.11,60


In [63]:
ucr_match.insert(0,'Key', range(1, 1+len(ucr_match)))

In [64]:
ucr_match.drop(ucr_match.columns[1], axis = 1, inplace = True)

In [65]:
ucr_match

,Key,Child Scores
0,1,100
1,2,100
2,3,100
3,4,100
4,5,20
...,...,...
1594,1595,40
1595,1596,20
1596,1597,20
1597,1598,60


# First iteration code below

In [29]:
#final_ucr_scores = pd.read_csv("315.csv")

# After Joining

In [66]:
#final_ucr_scores = pd.merge(h1,ucr_match, on = 'Key')
final_ucr_scores = pd.merge(final_ucr_scores, ucr_match, on = 'Key')

In [67]:
final_ucr_scores

,Unnamed: 0,Unnamed: 0.1,Key,Offense_Literal (Harris county),Murder Scores,Assault Scores,Rape Scores,Robbery Scores,Arson Scores,Burglary Scores,Fraud Scores,Weapon Scores,Theft Scores,Prostitute Scores,Liquor Scores,Drug Scores,Motor/Vehicle Scores,Gambling Scores,Drunk Scores,Child Scores
0,0,0,1,ABAN/ENDANG CHILD W/INT TO RET22.04,40,25,50,40,50,29,20,50,25,40,50,50,40,50,50,100
1,1,1,2,ABAN/ENDANG CHILD W/INT TO RET22.041,40,25,50,40,50,29,20,50,25,40,50,50,40,50,50,100
2,2,2,3,ABANDON/ENDANG CHILD W/O INT T22.041,40,25,25,40,50,29,40,25,25,20,33,50,40,33,50,100
3,3,3,4,ABANDONING CHILD UNDER 15 22.041,60,25,25,20,50,29,20,25,25,20,33,25,40,33,50,100
4,4,4,5,ABUSE OF OFF CAP>=2500 <30K,40,50,50,20,50,29,40,25,50,20,33,25,20,17,25,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,1594,1594,1595,WATER POLL ST LIABIL VIOL INDV,20,50,50,40,50,29,20,50,50,60,50,33,30,50,33,40
1595,1595,1595,1596,WATER POLLUTION 26.212,20,50,50,40,50,14,20,50,50,40,50,25,30,50,25,20
1596,1596,1596,1597,WEAPON-POSSESSION OF 46.02,20,25,50,40,50,14,20,75,25,40,33,0,30,33,25,20
1597,1597,1597,1598,WEAPON FREE SCHOOL ZONE-PROH W46.11,40,25,50,40,50,14,40,75,25,40,33,25,40,33,25,60


In [68]:
final_ucr_scores.to_csv('16 UCR Groups')